In [1]:
import os 
%pwd

'd:\\Data_analytics_new\\FSDS\\learning\\DS_Project\\DS-Project\\research'

In [2]:
os.chdir("../")
%pwd

'd:\\Data_analytics_new\\FSDS\\learning\\DS_Project\\DS-Project'

In [3]:
import pandas as pd

df = pd.read_csv("artifacts/data_ingestion/WineQT.csv")

In [4]:
df.head()

,fixed acidity,volatile acidity,citric acid,residual sugar,chlorides,free sulfur dioxide,total sulfur dioxide,density,pH,sulphates,alcohol,quality
0,7.4,0.70,0.00,1.9,0.076,11.0,34.0,0.9978,3.51,0.56,9.4,5
1,7.8,0.88,0.00,2.6,0.098,25.0,67.0,0.9968,3.20,0.68,9.8,5
2,7.8,0.76,0.04,2.3,0.092,15.0,54.0,0.9970,3.26,0.65,9.8,5
3,11.2,0.28,0.56,1.9,0.075,17.0,60.0,0.9980,3.16,0.58,9.8,6
4,7.4,0.70,0.00,1.9,0.076,11.0,34.0,0.9978,3.51,0.56,9.4,5


In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1143 entries, 0 to 1142
Data columns (total 12 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   fixed acidity         1143 non-null   float64
 1   volatile acidity      1143 non-null   float64
 2   citric acid           1143 non-null   float64
 3   residual sugar        1143 non-null   float64
 4   chlorides             1143 non-null   float64
 5   free sulfur dioxide   1143 non-null   float64
 6   total sulfur dioxide  1143 non-null   float64
 7   density               1143 non-null   float64
 8   pH                    1143 non-null   float64
 9   sulphates             1143 non-null   float64
 10  alcohol               1143 non-null   float64
 11  quality               1143 non-null   int64  
dtypes: float64(11), int64(1)
memory usage: 107.3 KB


In [6]:
df.isnull().sum()

fixed acidity           0
volatile acidity        0
citric acid             0
residual sugar          0
chlorides               0
free sulfur dioxide     0
total sulfur dioxide    0
density                 0
pH                      0
sulphates               0
alcohol                 0
quality                 0
dtype: int64

In [7]:
from dataclasses import dataclass
from pathlib import Path

@dataclass
class DataValidationConfig:
    root_dir:Path
    STATUS_FILE: str
    unzip_data_dir: Path
    all_schema: dict

In [8]:
from src.ds_project.constants import *
from src.ds_project.utils.common import read_yaml, create_directories

In [9]:
class ConfigurationManager:
    def __init__(self, config_filepath=CONFIG_FILE_PATH,
                 params_file_path=PARAMS_FILE_PATH,
                 schema_file_path=SCHEMA_FILE_PATH):
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_file_path)
        self.schema = read_yaml(schema_file_path)

        #print(self.config.artifacts_root)

        create_directories(self.config.artifacts_root)

    def get_data_validation(self)-> DataValidationConfig:
        config=self.config.data_validation
        schema=self.schema.COLUMNS
        create_directories(config.root_dir)

        data_validation_config=DataValidationConfig(
            root_dir=config.root_dir,
            STATUS_FILE=config.STATUS_FILE,
            unzip_data_dir=config.unzip_data_dir,
            all_schema=schema
        )
        return data_validation_config



In [10]:
import os
from src.ds_project import logger

In [11]:
class DataValidation:
    def __init__(self, config: DataValidationConfig):
        self.config=config

    def Validate_all_columns(self) -> bool:
        try:
            validation_status = None

            data = pd.read_csv(self.config.unzip_data_dir)
            all_col = list(data.columns)
            all_schema = self.config.all_schema.keys()
            print(all_col)
            print(all_schema)
    
            for col in all_col:
                if col not in all_schema:
                    validation_status=False
                    with open(self.config.STATUS_FILE, 'w') as f:
                        f.write(f"Validation failed: Column not found in schema: {validation_status}")
                else:
                    validation_status=True
                    with open(self.config.STATUS_FILE, 'w') as f:
                        f.write(f"Validation passed: Column found in schema: {validation_status}")

            return validation_status
        except Exception as e:
            raise e


In [12]:
try:
    config=ConfigurationManager()
    data_validation_config = config.get_data_validation()
    data_validation = DataValidation(config=data_validation_config)
    data_validation.Validate_all_columns()
    #data_ingestion.extract_zip()
except Exception as e:
    raise e


<_io.TextIOWrapper name='config\\config.yaml' mode='r' encoding='utf-8'>
{'artifacts_root': ['artifacts'], 'data_ingestion': {'root_dir': ['artifacts/data_ingestion'], 'source_URL': 'https://raw.githubusercontent.com/dushyantverma22/data_set/main/WineQT.csv', 'local_data_file': 'artifacts/data_ingestion/WineQT.csv', 'unzip_dir': 'artifacts/data_ingestion'}, 'data_validation': {'root_dir': ['artifacts/data_validation'], 'unzip_data_dir': 'artifacts/data_ingestion/WineQT.csv', 'STATUS_FILE': 'artifacts/data_validation/status.txt'}}
[2024-10-21 20:29:53,111: INFO: common: yaml file: config\config.yaml loaded successfully]
<_io.TextIOWrapper name='params.yaml' mode='r' encoding='utf-8'>
{'key': 'value'}
[2024-10-21 20:29:53,113: INFO: common: yaml file: params.yaml loaded successfully]
<_io.TextIOWrapper name='schema.yaml' mode='r' encoding='utf-8'>
{'COLUMNS': {'fixed acidity': 'float', 'volatile acidity': 'float', 'citric acid': 'float', 'residual sugar': 'float', 'chlorides': 'float', '